In [11]:
import ndex2.client as nc
import io
import json
from IPython.display import HTML
from time import sleep

from ndex2.nice_cx_network import NiceCXNetwork
import ndex2

import random

#create an NDEx client object to access my accouont on the NDEx server

my_account="Jeffreyc"
my_password="Frenchfri3"

my_ndex=nc.Ndex2("http://public.ndexbio.org", my_account, my_password)
my_ndex.update_status()

#find networks by search keyword "tutorial" and return uuid
list_tutorial_networks=my_ndex.search_networks('tutorial')
print("%s netwoks forund." % (len(list_tutorial_networks)))
print ("\nNetwork uuids:\n")
for ns in list_tutorial_networks.get('networks'): print( " %s" % ns.get('externalId'))
    
# create a NiceCX network "tutorial_nice_cx_from_server" from NDEx networks, in order to add attribute on the nodes
print ('enter network uuid in NDEx networks')
x = input()
tutorial_nice_cx_from_server = ndex2.create_nice_cx_from_server(server='http://public.ndexbio.org', uuid=x)

#check the number of attributes of nodes before adding 'Score'
tutorial_nice_cx_from_server.print_summary()

#create a loop to go through all the nodes in the network 
#check if a 'Score' attribute is already existing in each node.
#add attribute "Score" to each node with a random value from 1 to 10

for id, node in tutorial_nice_cx_from_server.get_nodes() :
       if tutorial_nice_cx_from_server.get_node_attribute(id, 'Score') == None:         
              v = random.randint(1,10)
              tutorial_nice_cx_from_server.add_node_attribute(id, name='Score', values=v)
        
#check the number of attributes of nodes after adding 'Score'
tutorial_nice_cx_from_server.print_summary()

#upload the updated NDEx network to my account
uri_upload_message = tutorial_nice_cx_from_server.upload_to("http://public.ndexbio.org", my_account, my_password)
print(uri_upload_message)



3 netwoks forund.

Network uuids:

 c9243cce-2d32-11e8-b939-0ac135e8bacf
 4197cc9c-c4a6-11e4-bcc4-000c29cb28fb
 3bd76cf4-c4a1-11e4-bcc4-000c29cb28fb
enter network uuid in NDEx networks
c9243cce-2d32-11e8-b939-0ac135e8bacf
Name: Metabolism of RNA
Nodes: 276
Edges: 17458
Node Attributes: 551
Edge Attributes: 0

Name: Metabolism of RNA
Nodes: 276
Edges: 17458
Node Attributes: 827
Edge Attributes: 0

http://public.ndexbio.org/v2/network/70b2af98-34c9-11e9-9f06-0ac135e8bacf


In [ ]:
#!pip uninstall ndex2 -y
#!pip install ndex2 --no-cache-dir

In [14]:
#search my account and find the network I added attribute "Score"
uuid = uri_upload_message.rpartition('/')[-1]

#delete all the previous networks I own
#list_uuid = my_ndex.get_network_ids_for_user("Jeffreyc")
#indexval = 0
#for i in range(len(list_uuid)):
#    if (list_uuid[indexval] != uuid):
#        my_ndex.delete_network(list_uuid[indexval])
#    indexval += 1
    
    
# Query the network from my NDEx account, get small subset with about 20-100 edges
# NDEx client object can query a subset, NiceCXNetwork object cannot query a subset?

#Experiment until get on with about 20-100 edges
query_result_cx=my_ndex.get_neighborhood(x, 'XRN1')
#query_result_cx=my_ndex.get_neighborhood(uuid, 'XPO1')
                                       
for aspect in query_result_cx:
    if 'nodes' in aspect:
        number_of_nodes = len(aspect['nodes'])
    if 'edges' in aspect:
        number_of_edges = len(aspect['edges'])

print("Query result network contains %s nodes and %s edges." % (number_of_nodes, number_of_edges))

#Save the subset to my account
uri_subset = my_ndex.save_new_network(query_result_cx)
uuid_subset = uri_subset.rpartition('/')[-1]

print (uuid_subset,uri_subset)

#new_summary = my_ndex.get_network_summary(uuid_subset)
 
if new_summary.get("isValid"):
        print("New network has been validated by the server.\n")
       # print (new_summary)
else:
    print("failed\n")
    #print (new_summary)


#update network profile
network_profile={"name":"Jeffrey subset Network", "description":"a small subset about 20-100 edges", "version":"2.0"}
my_ndex.update_network_profile(uuid_subset, network_profile)    
my_ndex.make_network_public(uuid_subset)


Query result network contains 26 nodes and 314 edges.
d31c78fd-34c9-11e9-9f06-0ac135e8bacf http://public.ndexbio.org/v2/network/d31c78fd-34c9-11e9-9f06-0ac135e8bacf
New network has been validated by the server.



''

In [13]:
#get subset network information 
ns=my_ndex.get_network_summary(uuid_subset)
def summary2table(object):
    table = "<table>"
    for key, value in object.items():
        if key == "warnings":
            warning_list = ""
            for warning in value:
                warning_list += "%s<br>" % warning
            value = warning_list
        if key == "properties":
            property_table = "<table>"
            for property in value:
                property_table += "<tr>"
                property_table += "<td>%s</td><td>%s</td>" % (property.get("predicateString"), property.get("value"))
                property_table += "</tr>"
            property_table += "</table>"
            value = property_table
        table += "<tr>"
        table += "<td>%s</td><td>%s</td>" % (key, value)
        table += "</tr>"
    table += "</table>"
    return table

HTML(summary2table(ns))

ownerUUID,4fa21fc0-121c-11e9-bb6a-0ac135e8bacf
isReadOnly,False
subnetworkIds,[]
isValid,True
warnings,
isShowcase,False
isCertified,False
indexLevel,NONE
hasLayout,False
hasSample,False
visibility,PUBLIC
